# Edge Addition Algorithm - simple implementation example

<font size="3">Run time is around ~5 minutes with the default input. \
\
In this example we will use the E.A.A. model to build a low-connectivity DCA model. \
The information that we have about the training RNA family consists in: the sequence alignment and the consenus secondary structure (both trough the Covariance Model) and the 3D contacts trough the PDB file. </font> 


In [1]:
display(HTML("<style>.container { width:90% !important; }</style>"))
using FCSeqTools
#include("plotting_tools.jl")

HTML{String}("<style>.container { width:90% !important; }</style>")

<font size="3">Here is an example of  RF00379 molecule and its associated consensus secondary structure. \
To make the execution faster we will not generate full lenght molecules but just a portion from nucleotide 55 to 102. </font>

In [2]:
first=55
last=102

tertiary_contact_list,tertiary_contact_matrix=tertiary_plot_tools(len,"mapp_4qln_A.pdb_A_RF00379.txt.csv",8);
natural_sequences=do_number_matrix_rna(do_letter_matrix("RF00379.txt"),0.2)
ss_contact_list, ss_contact_matrix=secondary_structure_plot_tools("RF00379.cm")
natural_sequences=do_number_matrix_rna(do_letter_matrix("RF00379.txt"),0.2)
natural_sequences=do_number_matrix_rna(do_letter_matrix("RF00379.txt")[:,first:last],0.2)
ss_contact_matrix=ss_contact_matrix[first:last,first:last]
tertiary_contact_matrix=tertiary_contact_matrix[first:last,first:last]
len=last-first+1;
println("The segment is $len nucletodide long and has $(sum(vec(ss_contact_matrix))) secondary structure contacts. The database contains $(length(natural_sequences[:,1])) sequences.")
println("\n", uppercase(join(collect(molecule)[first:last])))
println("\n", ss_matrix_to_dot_bracket(ss_contact_matrix))

UndefVarError: UndefVarError: tertiary_plot_tools not defined

<font size="3">Here is a segment example with its associated secondary structure. \
The database has a different size because the data-cleaning procedure depends on the region selected. \
Now we will run the E.A.A. building up our ineraction netwotk edge by edge till we reach a good performance generative model. \
At each iteration the algorithm reports: the added edge, the iteration number, the number of total added edges and the connectivity percentace of the fully connected case.\
Each 15 iterations the algorithm reports: the model score (Pearson between natural and artificial two-point correlations), the model mean energy, the model partition function and the model entropy. 

In [4]:
using Random
s=time()
Random.seed!(2) 
natural_sequences=do_number_matrix_rna(do_letter_matrix("RF00379.txt"),0.2)               #21       #12000
score,likelihood_gain,generated_sequences,Jij,h,contact_list,site_degree,edge_list = E_A_A(5, 0.05, 7000, natural_sequences,"example_output.txt");
s=time()-s

Fully connected model has 9180 edges and a score around ~ 0.95



[62  73]  iter: 1   edges: 

1   complex: 0.01%


[61  74]  iter: 2   edges: 2   complex: 0.02%


[63  72]  iter: 3   edges: 3   complex: 0.03%

...


[65  73]  iter: 1497   edges: 1399   complex: 15.24%


[64  126]  iter: 1498   edges: 1400   complex: 15.25%


[44  47]  iter: 1499   edges: 1401   complex: 15.26%


[44  69]  iter: 1500   edges: 1402   complex: 15.27%

InterruptException: InterruptException:

<font size="3">The model obtained has a performance comparable to the fully connected DCA while having just ~20% of its connectivity. The entropy of the model is 35.08. This means that it is able to generate e³⁵ (3.5x10¹⁵) different 55-102 segments for the RF00379 family. \
Now we can test our artificial sequences. We do the classical statistical check of the PCA projection and the two-point correlation representation. \
We test the performance of our model against the one of the Covariance Model. The CM model only contains trivial one-point and secondary information so our model must do better than it. </font>

In [ ]:
cm_sequences=rna_cm_model_generation(0.8,0.05,7000,natural_sequences,ss_contact_matrix);

In [ ]:
plot_stat_check(natural_sequences, generated_sequences, cm_sequences)

<font size="3">The E.A.A. artificial molecules are practically statistically indistinguishable from the natural ones. We see that they have a very similar PCA projection (artificial one seems richer just because we have more artificial sequences than natural ones) while Covariance Model fails to capture the details of the distribution. 
    The selected model has almost a perfect two-point statistics for all site pairs while the CM model only captures it for the ones involved in secondary structure contacts. \
     </font>


<font size="3">The interpretability is one of the main reasons in our quest to find parsimonious generative models. Now that we are sure we obtained a good generative model with relatively few parameters we can try to interprete them. \
Dividing the added edges in secondary structure contacts, 3D contacts we have:

In [ ]:
edge_interpretation_plot(len,ss_contact_matrix,tertiary_contact_matrix,edge_list[1:50,:])

<font size="3">We see that the secondary structure contacts are taken in the first few iteration. We have lot of neighbouring sites probably due to philogenic effects. It is striking that we see some 3D contacts (in particular around site 40) before the NONE edges. This
suggests that our algorithm effectively captures some information about the tertiary structure. \
Those results, that are far more general than this simple example, suggest that the added edges have a co-evolutionary interpretation.

<font size="3">
This notebook serves as an example of the application of the techniques described in the main text.
